In [ ]:
from io import BytesIO

import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [ ]:
import os
path = '../data'
files = os.listdir(path)
path_files = [path + '/'+ string for string in files]
file_entries = dict(zip(files,path_files))
file_entries

In [ ]:
select = pn.widgets.Select(options=file_entries)

@pn.cache
def fetch_data(url):
    df = pd.read_csv(url)
    df.drop('time', axis = 1, inplace = True)
    df.insert(0,'time',df.index.values)
    df = df.fillna(0)
    return df

select_df = pn.Column(select, pn.bind(pn.widgets.Tabulator, pn.bind(fetch_data, select), page_size=10))

In [ ]:
@pn.cache
def fetch_size(url):
    df = pd.read_csv(url)
    return df.size

In [ ]:
idf = hvplot.bind(fetch_data, select).interactive()
dir(idf)
idf.shape[0]

In [ ]:
png_pane_diagnostic = pn.pane.PNG('./pictures/diagnostic.png')


In [ ]:
png_pane = pn.pane.PNG('./pictures/pikachu.png')
png_pane

## Slide to see only a region


In [ ]:
time_slide_l = (
    pn.widgets.IntSlider(
        name = "Time slider",
        start = 0,
        value = 0,
        end = 2561,
        step = 1    )
)

In [ ]:
time_slide_r= (
    pn.widgets.IntSlider(
        name = "Time slider",
        start = 0,
        step = 1,
        value = 2561,
        end = 2561
        )
)

In [ ]:
time_slide_l_knee = (
    pn.widgets.IntSlider(
        name = "Time slider",
        start = 0,
        value = 0,
        end = 2561,
        step = 1    )
)

In [ ]:
time_slide_r_knee= (
    pn.widgets.IntSlider(
        name = "Time slider",
        start = 0,
        step = 1,
        value = 2561,
        end = 2561
        )
)

In [ ]:
# Radio buttons for CO2 measures
yaxis_shoulder = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['shoulderLangle', 'shoulderRangle',],
    button_type='success'
)

In [ ]:
time_slide_r_var = 1
dir(time_slide_l)

In [ ]:

shoulder_pipeline = (
    idf[(idf.time >= time_slide_l) &(idf.time <= time_slide_r) ])

In [ ]:
shoulder_pipeline

In [ ]:
shoulder_plot = shoulder_pipeline.hvplot(x = 'time',  y=yaxis_shoulder,line_width=2,  width=900, height=800, title="Shoulder Plot")
shoulder_plot


In [ ]:
yaxis_knee = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['kneeLangle', 'kneeRangle',],
    button_type='success'
)

In [ ]:
knee_pipeline = (
    idf[(idf.time >= time_slide_l_knee) &(idf.time <= time_slide_r_knee) ])

In [ ]:
knee_plot = shoulder_pipeline.hvplot(x = 'time',  y=yaxis_knee,line_width=2,  width=900, height=800, title="Knee Plot")
shoulder_plot

In [21]:
def df_pipeline(dataframe):
    return pn.pane.Markdown(dataframe)
df_name = pn.bind(df_pipeline, select)

In [23]:
df_string = pn.bind(lambda select: f'{select = }',select)
df_string

<function panel.depends._param_bind.<locals>.wrapped(*wargs, **wkwargs)>

In [25]:
html = '''
<!DOCTYPE HTML>
<html>
<head>
<meta charset="UTF-8">
<style type="text/css">
body {
	background-color: #f6f6ff;
	font-family: Calibri, Myriad;
}

#main {
	width: 780px;
	padding: 20px;
	margin: auto;
}

table.timecard {
	margin: auto;
	width: 600px;
	border-collapse: collapse;
	border: 1px solid #fff; /*for older IE*/
	border-style: hidden;
}

table.timecard caption {
	background-color: #150cc1;
	color: #fff;
	font-size: x-large;
	font-weight: bold;
	letter-spacing: .3em;
}

table.timecard thead th {
	padding: 8px;
	background-color: #9babc6;
	font-size: large;
}

table.timecard thead th#thDay {
	width: 40%;	
}

table.timecard thead th#thRegular, table.timecard thead th#thOvertime, table.timecard thead th#thTotal {
	width: 20%;
}

table.timecard th, table.timecard td {
	padding: 3px;
	border-width: 1px;
	border-style: solid;
	border-color: #4654f7 #ccc;
}

table.timecard td {
	text-align: right;
}

table.timecard tbody th {
	text-align: left;
	font-weight: normal;
}

table.timecard tfoot {
	font-weight: bold;
	font-size: large;
	background-color: #bac5df;
	color: #fff;
}

table.timecard tr.even {
	background-color: #46a4f7;
}
</style>
<title>Session Data</title>
</head>

<body>
<div id="main">
<table class="timecard">
	<caption>Session Data</caption>
	<thead>
		<tr>
			<th id="thMean">Mean</th>
			<th id="thVariance">Variance</th>
			<th id="thOvertime">Overtime</th>
		</tr>
	</thead>
	<tbody>
		<tr class="odd">
			<th>Shoulder</th>
			<td>8</td>
			<td>0</td>
		</tr>
		<tr class="even">
			<th>Sholder X</th>
			<td>8</td>
			<td>2.5</td>
		</tr>
		<tr class="odd">
			<th>Shoulder Y</th>
			<td>8</td>
			<td>0</td>
		</tr>
		<tr class="even">
			<th>Shoulder Z</th>
			<td>8</td>
			<td>0</td>
		</tr>
		<tr class="odd">
			<th>Knee</th>
			<td>8</td>
			<td>0</td>
		</tr>
		<tr class="even">
			<th>Knee X</th>
			<td>0</td>
			<td>5</td>
		</tr>
		<tr class="odd">
			<th>Knee Y</th>
			<td>0</td>
			<td>1</td>
		</tr>
	</tbody>

</table>
</div>
</body>
</html>
'''

In [26]:
html_pane = pn.pane.HTML(html)

In [24]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Wearable Device for Immersive Virtual Reality Control andApplication in Upper Limbs Motor Rehabilitation dashboard', 
    sidebar=[pn.pane.Markdown(""), 
             pn.pane.Markdown("#### Some meaningful instructions."), 
             select_df,
             png_pane_diagnostic,
             pn.pane.Markdown("## Settings"),
             pn.Row(pn.Column(time_slide_l, time_slide_r,))
            ],
    main=[pn.Row(pn.Column(yaxis_shoulder,
                           shoulder_plot.panel(width=700), margin=(0,25)), 
                  pn.Column(yaxis_knee,
                           knee_plot.panel(width=700), margin=(0,25))), 
          pn.Row(pn.Column(html_pane), 
                  png_pane)],
    accent_base_color="#87d8b1",
    header_background="#85d8b0",
)
# template.show()
template.servable();

NameError: name 'png_pane_diagnostic' is not defined